## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [1]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pydrake.all import MathematicalProgram, Variables, Polynomial, Solve
from pydrake.examples.pendulum import PendulumParams

In [3]:
# System dimensions. Here:
# x = [theta, theta_dot]
# z = [sin(theta), cos(theta), theta_dot]
nx = 2
nz = 3
nu = 1

# Map from original state to augmented state.
# Uses sympy to be able to do symbolic integration later on.
from sympy import sin, cos
x2z = lambda x : np.array([sin(x[0]), cos(x[0]), x[1]])

# System dynamics in augmented state (z).
params = PendulumParams()
inertia = params.mass() * params.length() ** 2
tau_g = params.mass() * params.gravity() * params.length()
def f(z, u):
    return [
        z[1] * z[2],
        - z[0] * z[2],
        (tau_g * z[0] + u[0] - params.damping() * z[2]) / inertia
    ]

# State limits (region of state space where we approximate the value function).
x_max = np.array([np.pi, 2*np.pi])
x_min = - x_max

# Equilibrium point in both the system coodinates.
x0 = np.array([0, 0])
z0 = x2z(x0)
    
# Quadratic running cost in augmented state.
Q = np.diag([1, 1, 1])
R = np.diag([5])
def l(z, u):
    return (z - z0).dot(Q).dot(z - z0) + u.dot(R).dot(u)

In [9]:
from sympy import Symbol, integrate
# Function that integrates a (multivariate) function  p(x2z(x)) over the interval [x_min, x_max],
# p(z) is assumed to be a polynomial. Needed by the objective functions in the SOS program.
def polyint(p, x_min, x_max, x2z=None):

    # integration variables
    nx = len(x_min)
    assert(len(x_max) == nx)
    x = [Symbol(f'x({i})') for i in range(nx)]

    # evaluate transformation if provided
    z = x if x2z is None else x2z(x)

    # compute integral one monomial per time
    integral = 0
    for m, c in p.monomial_to_coefficient_map().items():

        # integrand for the current monomial
        m_integrand = 1
        for i, zi in enumerate(p.indeterminates()):
            m_integrand *= z[i] ** m.degree(zi)

        # numeric value of the integral of the monomial
        m_integral = m_integrand
        for i, x_i in enumerate(x):
            m_integral = integrate(m_integral, (x_i, x_min[i], x_max[i]))

        # add monomial integral to the overall polynomial integral
        integral += c * float(m_integral)

    return integral

In [10]:
# Given the degree for the approximate value function and the polynomials
# in the S procedure, solves the SOS and returns the approximate value function
# (together with the objective of the SOS program).
def approximate_dp(deg):
    
    # Set up optimization.
    prog = MathematicalProgram()
    z = prog.NewIndeterminates(nz, 'z')
    u = prog.NewIndeterminates(nu, 'u')
    v = prog.NewFreePolynomial(Variables(z), deg)
    v_expr = v.ToExpression()

    # Maximize volume beneath the value function.
    obj = polyint(v, x_min, x_max, x2z)
    prog.AddLinearCost(- obj)

    # S procedure for s^2 + c^2 = 1.
    lam = prog.NewFreePolynomial(Variables(z), deg).ToExpression()
    S_procedure = lam * (z[0]**2 + z[1]**2 - 1)

    # Enforce Bellman inequality.
    v_dot = v_expr.Jacobian(z).dot(f(z, u))
    prog.AddSosConstraint(v_dot + l(z, u) + S_procedure)

    # v(z0) = 0.
    v0 = v_expr.EvaluatePartial(dict(zip(z, z0)))
    prog.AddLinearConstraint(v0 == 0)
    
     # Solve and retrieve result.
    result = Solve(prog)
    assert result.is_success()
    v_star = Polynomial(result.GetSolution(v_expr))

    # Solve for the optimal feedback in augmented coordinates.
    Rinv = np.linalg.inv(R)
    f2 = np.array([[0], [0], [1 / inertia]])
    dvdz = v_star.ToExpression().Jacobian(z)
    u_star = - .5 * Rinv.dot(f2.T).dot(dvdz.T)
    
    return z, u_star, v_star

In [ ]:
from pydrake.all import DiagramBuilder, Simulator, VectorSystem, WrapToSystem, LogOutput
from pydrake.examples.pendulum import PendulumPlant
from underactuated.pendulum import PendulumVisualizer
from underactuated.jupyter import AdvanceToAndVisualize

offset = np.array([np.pi, 0])
def simulate(controller, x0):
  
    plant = PendulumPlant()
    simulator = Simulator(plant)
    
    builder = DiagramBuilder()
    pendulum = builder.AddSystem(PendulumPlant())

    wrap = builder.AddSystem(WrapToSystem(2))
    wrap.set_interval(0, 0, 2*np.pi)
    builder.Connect(pendulum.get_output_port(0), wrap.get_input_port(0))
    controller = builder.AddSystem(controller)
    builder.Connect(wrap.get_output_port(0), controller.get_input_port(0))
    builder.Connect(controller.get_output_port(0), pendulum.get_input_port(0))

    logger = LogOutput(pendulum.get_output_port(0), builder)

    visualizer = builder.AddSystem(PendulumVisualizer(show=False))
    builder.Connect(pendulum.get_output_port(0), visualizer.get_input_port(0))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_context().SetContinuousState(x0 + offset)

    duration = 5
    visualizer.start_recording()
    simulator.Initialize()
    simulator.AdvanceTo(duration)
    visualizer.stop_recording()

    return logger, visualizer.get_recording_as_animation()

def solve_and_simulate(deg, x0):
    
    # solve SOS
    z, ustar, Jstar = approximate_dp(deg)
    
    # get controller in original coordinates
    def state_feedback(x):
        env = {z[0]: np.sin(x[0]), z[1]: np.cos(x[0]), z[2]: x[1]}
        return np.array([ui.Evaluate(env) for ui in ustar])
    
    # drake controller
    class Controller(VectorSystem):
        def __init__(self, state_feedback):
            VectorSystem.__init__(self, 2, 1)
            self.state_feedback = state_feedback
        def DoCalcVectorOutput(self, context, x, unused, u):
            u[:] = self.state_feedback(x - offset)
            
    # run sim
    return simulate(Controller(state_feedback), x0)

In [ ]:
x0 = np.array([.9 * np.pi, 0])
logger, ani = solve_and_simulate(5, x0)

In [ ]:
from matplotlib import animation
from IPython.display import HTML, display
HTML(ani.to_jshtml())